In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [12]:
cd /content/drive/MyDrive/PaddleOCR

/content/drive/MyDrive/PaddleOCR


In [13]:
!pip install paddlepaddle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
cd /content/drive/MyDrive/PaddleOCR/PaddleOCR

/content/drive/MyDrive/PaddleOCR/PaddleOCR


In [15]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import cv2
import os
import time
import numpy as np
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img

In [4]:
cd /content/drive/MyDrive/PaddleOCR/PaddleOCR

/content/drive/MyDrive/PaddleOCR/PaddleOCR


In [5]:
from tools.infer.predict_rec import *
import tools.infer.utility as utility
from ppocr.postprocess import build_post_process
from ppocr.utils.logging import get_logger
from ppocr.utils.utility import get_image_file_list, check_and_read_gif

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):


In [6]:
def rec(args, out_path, input, rec_model_dir,rec_image_shape = "3, 32, 320", rec_char_type = "en", rec_algorithm = "CRNN", show = True, save = True):
    # Assigning values to args as the code is not running from console.
    args.rec_model_dir = rec_model_dir
    args.rec_image_shape = rec_image_shape
    args.rec_char_type = rec_char_type
    args.rec_algorithm = rec_algorithm
    

    # Initializing some helper variables.
    t1 = 0
    t2 = 0
    tot = []
    os.chdir('./PaddleOCR')

    
    # Initializing recognizer.
    image_file_list = get_image_file_list(input)
    text_recognizer = TextRecognizer(args)
    valid_image_file_list = []
    img_list = []

    # Warming up GPU to run it at its full capacity.
    if args.warmup:
        image = np.random.uniform(0, 255, [32, 320, 3]).astype(np.uint8)
        for i in range(10):
            res = text_recognizer([image])

    # Đọc và thêm ảnh vào . 
    for image_file in image_file_list:
        image, flag = check_and_read_gif(image_file)
        if not flag:
            image = cv2.imread(image_file)
        if image is None:
            logger.info("error in loading image:{}".format(image_file))
            continue
        valid_image_file_list.append(image_file)
        img_list.append(image)

    # OCR images.
    t1 = time.time()
    try:
        rec_res, _ = text_recognizer(img_list)
    except Exception as E:
        logger.info(traceback.format_exc())
        logger.info(E)
        exit()
    
    # Tính  FPS và hiển thị.
    t2 = time.time()
    fps = str(t2-t1)
    for ino in range(len(img_list)):
        logger.info("Predicts of {}:{}".format(valid_image_file_list[ino],
                                              rec_res[ino]))
        if save:
          cv2.imwrite(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '_rec' + '.jpg'),img_list[ino])
          with open(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '.txt'), 'w') as f:
            f.write(str(rec_res[ino]))
    logger.info("Time taken recognize all images : {}".format(fps))
    print(len(image_file_list))
    logger.info("Average fps : {}".format(1/(float(fps)/len(image_file_list))))
    
    if show:
      plt.figure(figsize = (25,14))
      plt.imshow(image)
      plt.show()

In [7]:
%cd /content/drive/MyDrive/PaddleOCR
out_path = '/content/drive/MyDrive/PaddleOCR/val_out'
rec_model_dir = '/content/drive/MyDrive/PaddleOCR/PaddleOCR/output/inference'
input = '/content/drive/MyDrive/PaddleOCR/data/val'
sys.argv = ['']
rec(utility.parse_args(), out_path, input, rec_model_dir, show = False)

/content/drive/MyDrive/PaddleOCR
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_41.png:('18:45', 0.9183559417724609)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_42.png:('9:00', 0.8100190162658691)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_43.png:('16:00', 0.7357887029647827)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_44.png:('17:00', 0.8723136186599731)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_45.png:('7:10', 0.909076452255249)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_46.png:('8:45', 0.9550472497940063)
[2022/08/01 09:17:13] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/data/val/Screenshot_47.png:('15:00', 0.991582989692688)
[2022/08/01 09:17:13

In [8]:
%cd /content/drive/MyDrive/PaddleOCR
out_path = '/content/drive/MyDrive/PaddleOCR/break_time_out'
rec_model_dir = '/content/drive/MyDrive/PaddleOCR/PaddleOCR/output/inference'
input = '/content/drive/MyDrive/PaddleOCR/break_time'
sys.argv = ['']
rec(utility.parse_args(), out_path, input, rec_model_dir, show = False)

/content/drive/MyDrive/PaddleOCR
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/104.jpg:('8000', 0.32733720541000366)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/120.jpg:('80', 0.613327145576477)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/123.jpg:('7:0', 0.5912773013114929)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/136.jpg:('F8', 0.6058065891265869)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/142.jpg:(':0', 0.5933154225349426)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/152.jpg:('80', 0.6765602827072144)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_time/161.jpg:('1:0', 0.7400164008140564)
[2022/08/01 09:39:52] ppocr INFO: Predicts of /content/drive/MyDrive/PaddleOCR/break_tim

In [9]:
def rec2(args, out_path, input, rec_model_dir,rec_image_shape = "3, 32, 320", rec_char_type = "en", rec_algorithm = "CRNN", show = True, save = True):
    # Assigning values to args as the code is not running from console.
    args.rec_model_dir = rec_model_dir
    args.rec_image_shape = rec_image_shape
    args.rec_char_type = rec_char_type
    args.rec_algorithm = rec_algorithm
    

    # Initializing some helper variables.
    t1 = 0
    t2 = 0
    tot = []
    os.chdir('./PaddleOCR')

    
    # Initializing recognizer.
    image_file_list = get_image_file_list(input)
    text_recognizer = TextRecognizer(args)
    valid_image_file_list = []
    img_list = []

    # Warming up GPU to run it at its full capacity.
    if args.warmup:
        image = np.random.uniform(0, 255, [32, 320, 3]).astype(np.uint8)
        for i in range(10):
            res = text_recognizer([image])

    # Đọc và thêm ảnh vào . 
    for image_file in image_file_list:
        image, flag = check_and_read_gif(image_file)
        if not flag:
            image = cv2.imread(image_file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.threshold(cv2.medianBlur(image, 3), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        if image is None:
            logger.info("error in loading image:{}".format(image_file))
            continue
        valid_image_file_list.append(image_file)
        img_list.append(image)

    # OCR images.
    t1 = time.time()
    try:
        rec_res, _ = text_recognizer(img_list)
    except Exception as E:
        logger.info(traceback.format_exc())
        logger.info(E)
        exit()
    
    # Tính  FPS và hiển thị.
    t2 = time.time()
    fps = str(t2-t1)
    for ino in range(len(img_list)):
        logger.info("Predicts of {}:{}".format(valid_image_file_list[ino],
                                              rec_res[ino]))
        if save:
          cv2.imwrite(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '_rec' + '.jpg'),img_list[ino])
          with open(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '.txt'), 'w') as f:
            f.write(str(rec_res[ino]))
    logger.info("Time taken recognize all images : {}".format(fps))
    print(len(image_file_list))
    logger.info("Average fps : {}".format(1/(float(fps)/len(image_file_list))))
    
    if show:
      plt.figure(figsize = (25,14))
      plt.imshow(image)
      plt.show()

In [ ]:
%cd /content/drive/MyDrive/PaddleOCR
out_path = '/content/drive/MyDrive/PaddleOCR/break_time_cv2'
rec_model_dir = '/content/drive/MyDrive/PaddleOCR/PaddleOCR/output/inference'
input = '/content/drive/MyDrive/PaddleOCR/break_time'
sys.argv = ['']
rec2(utility.parse_args(), out_path, input, rec_model_dir, show = False)

In [3]:
def disp(pth, gt_annot = '', gt = False, out = False, num = 10):
  img_arr = []
  annot_arr = []
  
  # Appending image array into a list.
  for fimg in sorted((os.listdir(pth))):
    if fimg.endswith('.jpg') or fimg.endswith('.png'):
      demo = img.imread(pth+fimg) 
      img_arr.append(demo)
      
      # Appending dataset output into a list (OCR outputs are stored in different text files for every image).
      if out:
        with open(pth + ''.join(fimg)[:-8]+ '.txt') as f:
          out = f. read()
          f.close()
        annot_arr.append(out.lower())
        
      # Appending ground truth annotations into a list (Ground truth of all images are stored in a single text file).
      if gt:
        with open(gt_annot) as f:
          for line in f:
            if line.split(',')[0][4:] == fimg: 
              gt = line.split(',')[1].lower()
              annot_arr.append(gt)
              break

      if len(img_arr) == num:
          break

  # Displaying the images along with labels.
  _, axs = plt.subplots(2, 5, figsize=(25, 14))
  axs = axs.flatten()
  for cent, ax,val  in zip(img_arr, axs, annot_arr):
      ax.imshow(cent)
      ax.set_title(val,fontsize=25)
  plt.show()